In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/slehmann1/Fastenating.git

Cloning into 'Fastenating'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 6), reused 31 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 278.67 KiB | 27.87 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
!pip install transformers llama-cpp-python langchain_community wikipedia
!pip install requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510582 sha256=60e1972281f6d1e56a97d204398cd6afb9c16ce04470c0d857218ae196b4f67f
  Stored in directory: /root/.cache/pip/wheels/f

In [ ]:
!pip uninstall numpy pandas scipy
!pip cache purge
!pip install numpy pandas scipy
!pip uninstall transformers
!pip install transformers

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.10/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.re

In [ ]:
import torch
import transformers
import llama_cpp
import langchain_community.utilities.wikipedia
import requests
from typing import Dict, Any
import sys
sys.path.append("/content/Fastenating")
import fastener_toolkit as ft
import visualizer as vis
import math

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
llm = llama_cpp.Llama.from_pretrained(
    repo_id="Qwen/Qwen2.5-7B-Instruct-GGUF",
    filename="qwen2.5-7b-instruct-q2_k.gguf",
    n_ctx=16000,
    device = device
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


qwen2.5-7b-instruct-q2_k.gguf:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct-GGUF/snapshots/bb5d59e06d9551d752d08b292a50eb208b07ab1f/./qwen2.5-7b-instruct-q2_k.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-7b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-7b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 7.6B
llama_model_loader: - kv   6

In [ ]:
def llm_engine(messages, stop_sequences=["Task"]) -> str:

    response = llm.create_chat_completion(
        messages = messages,
        stop=stop_sequences,
        max_tokens=10000,
    )

    answer = response["choices"][0]["message"]["content"]

    return answer

In [ ]:
class FastenatingCalculator(transformers.Tool):
  name = "Yield_Safety_Of_Factor_Calculation"
  description = "Calculates the factor of safety for fasteners using Yield Strength."

  inputs = {
        "c": {
            "type": "number",
            "description": "Joint Constant in N/mm",
        },
        "d_major": {
            "type": "number",
            "description": "Major diameter of the bolt in mm",
        },
        "d_minor": {
            "type": "number",
            "description": "Minor diameter of the bolt in mm",
        },
        "load": {
            "type": "number",
            "description": "Load applied to the bolt in Newtons",
        },
        "yield_strength": {
            "type": "number",
            "description": "Yield strength of the bolt material in MPa",
        },
        "preload": {
            "type": "number",
            "description": "Preload applied to the bolt in Newtons",
        },
        # "E_b": {
        #     "type": "number",
        #     "description": "Young's modulus of the bolt in MPa or psi",
        # },
        # "E_m": {
        #     "type": "number",
        #     "description": "Young's modulus of the material in MPa or psi",
        # },
        # "l_unthreaded": {
        #     "type": "number",
        #     "description": "Length of unthreaded portion within the grip zone in mm",
        # },
        # "l_threaded": {
        #     "type": "number",
        #     "description": "Length of threaded portion within the grip zone in mm",
        # },
        # "s_ut": {
        #     "type": "number",
        #     "description": "Ultimate tensile strength of the bolt in MPa",
        # },
        # "s_end": {
        #     "type": "number",
        #     "description": "Endurance limit of the bolt in MPa or psi",
        # }
    }

  output_type = "text"

  def __init__(self):
        self.ft = ft
        # self.vis = vis

  def cal_tensile_area(self,d_major,d_minor):
    area = (math.pi / 4) * ((d_major + d_minor)/2)**2
    return area

  def forward(
    self,
    c: float,
    d_major: float,
    d_minor: float,
    load: float,
    yield_strength: float,
    preload: float) -> str:

    try:
      tensile_area = self.cal_tensile_area(d_major, d_minor)
      sf = self.ft.bolt_yield_safety_factor(c=c,load=load,preload=preload,a_ts = tensile_area,b_ys = yield_strength)
      return sf
    except requests.RequestException as e:
        return f"Error occurred while fetching data: {str(e)}"
    except KeyError:
        return "Unable to calculate the number of fasteners. Please check the input values."

    # # # Detect whether the user wants to generate a plot
    # # plot = "plot" in query.lower() or "generate a plot" in query.lower()

    # # Combine inputs into a params dictionary for easier handling in the LLM
    # params = {
    #     "d_major": d_major,
    #     "d_minor": d_minor,
    #     "load": load,
    #     "yield_strength": yield_strength,
    #     "preload": preload
    #     # "E_b": E_b,
    #     # "E_m": E_m,
    #     # "l_unthreaded": l_unthreaded,
    #     # "l_threaded": l_threaded,
    #     # "s_ut": s_ut,
    #     # "s_end": s_end
    # }

    # # Formulating the context for the LLM
    # context = {
    #     "fastener_toolkit": self.ft,
    #     "visualizer": self.vis,
    #     "params": params,
    #     "plot": plot,
    #     "image_path": "plot_image.png" if plot else None
    # }

    # # Using LLM to interpret the query and dynamically call functions from fastener_toolkit
    # response = self.llm_engine({
    #     "task": "Calculate factor of safety and generate a plot if requested.",
    #     "query": query,
    #     "context": context
    # })


    # return response

In [ ]:
ft.bolt_yield_safety_factor

<function fastener_toolkit.bolt_yield_safety_factor(c, load, preload, a_ts, b_ys)>

In [ ]:
agent = transformers.ReactJsonAgent(
    tools=[FastenatingCalculator()],
    llm_engine = llm_engine,
    add_base_tools=False,
    verbose=2,
    max_iterations = 1
)

In [ ]:
agent.run("Calculate the factor of safety for a steel bolt with the following specifications: joint constant: 0.25 N/mm, major diameter of 16 mm, minor diameter of 13.5 mm, preload of 15000 N, applied load of 10000 N, yield strength of 350 MPa")

======== New task ========
Calculate the factor of safety for a steel bolt with the following specifications: joint constant: 0.25 N/mm, major diameter of 16 mm, minor diameter of 13.5 mm, preload of 15000 N, applied load of 10000 N, yield strength of 350 MPa
System prompt is as follows:
You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: 'Yield_Safety_Of_Factor_Calculation', 'final_answer'
The way you use the tools is by specifying a json blob, ending with '<end_action>'.
Specifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:
{
  "action": $TOOL_NAME,
  "action_i

"The factor of safety (FoS) for the steel bolt can be calculated using the formula:\n\n\\[ \\text{FoS} = \\frac{\\text{Yield Strength}}{\\text{Allowable Stress}} \\]\n\nFirst, we need to calculate the allowable stress. The allowable stress is given by the formula:\n\n\\[ \\text{Allowable Stress} = \\left( \\frac{\\text{c} \\times \\left( \\text{d_major} - \\text{d_minor} \\right) \\times (\\text{preload} + \\text{load})}{\\pi \\times \\text{d_major} \\times \\text{d_minor}} \\right) + \\left( \\frac{\\text{c} \\times \\left( \\text{d_major} - \\text{d_minor} \\right) \\times \\text{preload}}{\\pi \\times \\text{d_major} \\times \\text{d_minor}} \\right) \\]\n\nWhere:\n- \\(\\text{c}\\) is the joint constant (0.25 N/mm),\n- \\(\\text{d_major}\\) is the major diameter (16 mm),\n- \\(\\text{d_minor}\\) is the minor diameter (13.5 mm),\n- \\(\\text{preload}\\) is the preload (15000 N),\n- \\(\\text{load}\\) is the applied load (10000 N).\n\nLet's calculate the allowable stress:\n\n\\[ \\te